# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)

cities_df = pd.read_csv("../WeatherPy/city_data.csv")
cities_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mahon,39.8885,4.2658,72.16,100,0,8.05,ES,1666474053
1,1,Longyearbyen,78.2186,15.6401,21.04,58,20,4.61,SJ,1666473481
2,2,Thompson,55.7435,-97.8558,41.16,56,75,4.61,CA,1666473477
3,3,Saint George,37.1041,-113.5841,79.79,17,0,33.38,US,1666473474
4,4,Butaritari,3.0707,172.7902,81.14,67,100,10.13,KI,1666473470
...,...,...,...,...,...,...,...,...,...,...
544,544,Linqiong,30.4159,103.4609,67.82,62,97,2.08,CN,1666474250
545,545,Marabu,4.4148,114.0089,77.14,94,40,0.00,MY,1666474251
546,546,Burnie,-41.0667,145.9167,53.94,96,100,13.56,AU,1666474251
547,547,Bathsheba,13.2167,-59.5167,84.81,74,40,12.66,BB,1666474252


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert humidity to float and store
humidity = cities_df["Humidity"].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Narrow down the cities to fit weather conditions.
#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
hotel_df = cities_df.loc[(cities_df["Max Temp"] < 80) & (cities_df["Max Temp"] > 75) & (cities_df["Wind Speed"] < 10) & (cities_df["Cloudiness"] == 0)]

#Drop any rows will null values.
hotel_df.dropna()

#print new dataframe 
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,Salalah,17.0151,54.0924,78.89,69,0,3.44,OM,1666473536
210,210,Bowmore,34.9421,-79.3031,76.51,46,0,8.05,US,1666474117
234,234,Dzaoudzi,-12.7887,45.2699,76.89,83,0,5.75,YT,1666474125
258,258,Mount Isa,-20.7333,139.5000,78.35,23,0,8.41,AU,1666473633
293,293,Sur,22.5667,59.5289,76.33,73,0,5.17,OM,1666473588
452,452,Russell,32.3502,-85.1999,79.02,25,0,3.44,US,1666474200
470,470,Rājula,21.0500,71.4333,77.94,59,0,7.47,IN,1666474217


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotels = []
hotel_df.append({"Hotel Name": hotels}, ignore_index=True)

# Use the lat/lng from cities to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # geocoordinates
    target_coordinates = f"{lat},{lng}"
    target_radius = 5000
    target_search = "hotel"
    
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    
    hotel_data = response.json()

    # Print the json (pretty printed)
    #print(json.dumps(hotel_data, indent=4, sort_keys=True))
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = hotel_data["results"][0]['name']    

    except Exception as e:
        print("Missing field/result... skipping.")  

/var/folders/lf/_fj3_9ln3j5bmwl546qpzm1w0000gn/T/ipykernel_37517/2258409672.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hotel_df.append({"Hotel Name": hotels}, ignore_index=True)
/var/folders/lf/_fj3_9ln3j5bmwl546qpzm1w0000gn/T/ipykernel_37517/2258409672.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df.loc[index, 'Hotel Name'] = hotel_data["results"][0]['name']


Missing field/result... skipping.
Missing field/result... skipping.


In [7]:
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
32,32,Salalah,17.0151,54.0924,78.89,69,0,3.44,OM,1666473536,Darbat Hotel
210,210,Bowmore,34.9421,-79.3031,76.51,46,0,8.05,US,1666474117,NaN
234,234,Dzaoudzi,-12.7887,45.2699,76.89,83,0,5.75,YT,1666474125,Hotel Maharajah Mayotte
258,258,Mount Isa,-20.7333,139.5000,78.35,23,0,8.41,AU,1666473633,ibis Styles Mt Isa Verona
293,293,Sur,22.5667,59.5289,76.33,73,0,5.17,OM,1666473588,Sur Grand Hotel
452,452,Russell,32.3502,-85.1999,79.02,25,0,3.44,US,1666474200,NaN
470,470,Rājula,21.0500,71.4333,77.94,59,0,7.47,IN,1666474217,Lion Lords Inn Rajula


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    locations, info_box_content= hotel_info
)

#add layers to fig
fig = gmaps.figure()
fig.add_layer(hotel_layer)
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))